In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import pickle
from dateutil.parser import *
import time
import logging
import os
import pytz
import pprint

In [2]:
API_KEY = 'x'
ACCOUNT_ID = 'x'
OANDA_URL = 'https://api-fxpractice.oanda.com/v3'

SECURE_HEADER = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

In [3]:
class OandaTrade():
    def __init__(self,oanda_ob):
        
        self.unrealizedPL = float(oanda_ob['unrealizedPL'])
        self.currentUnits = int(oanda_ob['currentUnits'])
        self.trade_id= int(oanda_ob['id'])
        self.openTime = parse(oanda_ob['openTime'])
        self.instrument = oanda_ob['instrument']
        
    def __repr__(self):
        return str(vars(self))

    @classmethod
    def TradeFromAPI(cls, api_object):
        return OandaTrade(api_object)

In [4]:
class OandaAPI():

    def __init__(self):
        self.session = requests.Session()    
    
    #connection link to server
    def make_request(self, url, params={}, added_headers=None, verb='get', data=None, code_ok=200):

        headers = SECURE_HEADER

        if added_headers is not None:   
            for k in added_headers.keys():
                headers[k] = added_headers[k]
                
        try:
            response = None
            if verb == 'post':
                response = self.session.post(url,params=params,headers=headers,data=data)
            elif verb == 'put':
                response = self.session.put(url,params=params,headers=headers,data=data)
            else:
                response = self.session.get(url,params=params,headers=headers,data=data)

            status_code = response.status_code

            if status_code == code_ok:
                json_response = response.json()
                return status_code, json_response
            
            else:
                return status_code, None  
        except:
            print('make_request error 1')
            try:
                response = None
                if verb == 'post':
                    response = self.session.post(url,params=params,headers=headers,data=data)
                elif verb == 'put':
                    response = self.session.put(url,params=params,headers=headers,data=data)
                else:
                    response = self.session.get(url,params=params,headers=headers,data=data)
                status_code = response.status_code
                
                if status_code == code_ok:
                    json_response = response.json()
                    return status_code, json_response
            
                else:
                    return status_code, None  
            except:
                print("make_request error 2")
                return 400, None   
    
    #get instruments list
    def fetch_instruments(self):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"
        status_code, data = self.make_request(url)
        return status_code, data
    
    #change instruments list to df
    def get_instruments_df(self):
        status_code, data = self.fetch_instruments()
        if status_code == 200:
            df = pd.DataFrame.from_dict(data['instruments'])
            return df[['name', 'type', 'displayName', 'pipLocation', 'marginRate']]
        else:
            return None
    
    #get candles from server and change to df with candles_to_df
    def fetch_candles(self, pair_name, count = 32,granularity="M1"):
        url = f"{OANDA_URL}/instruments/{pair_name}/candles"

        params = dict(
            granularity = granularity,
            price = "MBA"
        )
        
        params['count'] = count
        
        
        status_code, data = self.make_request(url, params=params)

        if status_code != 200:
            return status_code, None

        return status_code, OandaAPI.candles_to_df(data['candles'])
    
    #get last candle time
    def last_complete_candle(self,pair_name,granularity ='M1'):
        code, df = self.fetch_candles(pair_name,granularity = granularity)
        if df is None or df.shape[0]==0:
            return None
        return df.iloc[-1].time
    
    
    def place_trade(self,pair,units,take_profit = None, stop_loss = None):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/orders"

        data = {
            "order": {
                "units": units,
                "instrument": pair,
                "timeInForce": "FOK",
                "type": "MARKET",
                "positionFill": "DEFAULT"
            }
        }
        
        #make a trade with the data above
        status_code, json_data = self.make_request(url,verb = 'post',data = json.dumps(data), code_ok = 201)
        
        if status_code!=201:
            return None
        
        trade_id = None
        ok1 = True
        ok2= True
        
        #get trade id after making trade
        if 'orderFillTransaction' in json_data and 'tradeOpened' in json_data['orderFillTransaction']:
            trade_id =  int(json_data['orderFillTransaction']['tradeOpened']['tradeID'])
            price_str =  json_data['orderFillTransaction']['price']
            units = int(json_data['orderFillTransaction']['tradeOpened']['units'])
            print('trade done')
        
        #runs function to create tp after getting trade_id
        if take_profit is not None:
            if units>0:
                if (self.set_sl_tp(float(price_str)+take_profit, 'TAKE_PROFIT', trade_id) ==False):
                    ok1= False
            if units<0:
                if (self.set_sl_tp(float(price_str)-take_profit, 'TAKE_PROFIT', trade_id) ==False):
                    ok1= False                   
        if stop_loss is not None:
            if units>0:
                if (self.set_sl_tp(float(price_str)-take_profit, 'STOP_LOSS', trade_id) == False):
                    ok2=False
            if units<0:
                if (self.set_sl_tp(float(price_str)+take_profit, 'STOP_LOSS', trade_id) == False):
                    ok2=False
        return trade_id,ok1,ok2
                
    #function to make tp trades
    def set_sl_tp(self,price,order_type,trade_id):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/orders"
        data = {
            "order": {
                "timeInForce": "GTC",
                "price": str(price), 
                "type": order_type,
                "tradeID": str(trade_id)
            }
        }
        
        status_code, json_data = self.make_request(url, verb='post', data=json.dumps(data), code_ok=201)

        if status_code != 201:
            data = {
            "order": {
                "timeInForce": "GTC",
                "price": str(round(price,3)), 
                "type": order_type,
                "tradeID": str(trade_id)
            }
        }
            status_code, json_data = self.make_request(url, verb='post', data=json.dumps(data), code_ok=201)
            
            if status_code != 201:
                print (status_code)
                return False
        return True
    
    #run request to close trade
    def close_trade(self,trade_id):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/trades/{trade_id}/close"
        status_code, json_data = self.make_request(url, verb='put', code_ok=200)
        if status_code !=200:
            print(status_code)
            return False
        print ('close trade')
        return True
    
    #returns all open trades as OandaTrade object
    def open_trades(self):
        url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/openTrades"
        status_code, data = self.make_request(url)
        
        if status_code !=200:
            return [], False
        #could be that the code work but we dont have any open trades. will still return false. so return true as long as 
        #'trades' not in data
        if 'trades' not in data:
            return [], True
        
        trades = [OandaTrade.TradeFromAPI(x) for x in data['trades']]
        
        return trades,True
    
    #change candles from fetch candles to a dataframe
    @classmethod
    def candles_to_df(cls, json_data):
        prices = ['mid', 'bid', 'ask']
        ohlc = ['o', 'h', 'l', 'c']

        our_data = []
        for candle in json_data:
            if candle['complete'] == False:
                continue
            new_dict = {}
            new_dict['time'] = candle['time']
            new_dict['volume'] = candle['volume']
            for price in prices:
                for oh in ohlc:
                    new_dict[f"{price}_{oh}"] = float(candle[price][oh])
            our_data.append(new_dict)
        df = pd.DataFrame.from_dict(our_data)
        df["time"] = [parse(x) for x in df.time]
        return df

if __name__ == "__main__":
    api = OandaAPI()
    #res, df = api.fetch_candles("EUR_USD")
    #print(df)
    #print(api.last_complete_candle('EUR_USD'))
    trades, ok = api.open_trades()
    if ok == True:
         [print(t) for t in trades]
         print(ok)
#api=OandaAPI()

#api.set_sl_tp(89.54000001,'STOP_LOSS','248')

{'unrealizedPL': -1.3755, 'currentUnits': 1000, 'trade_id': 931, 'openTime': datetime.datetime(2021, 6, 25, 1, 1, 11, 549545, tzinfo=tzutc()), 'instrument': 'AUD_JPY'}
True


In [5]:
def time_utc():
    return dt.datetime.utcnow().replace(tzinfo=dt.timezone.utc)

def get_utc_dt_from_string(date_str):
    d = parse(date_str)
    return d.replace(tzinfo=dt.timezone.utc)

In [6]:
#create a logging file

LOG_FORMAT= '%(asctime)s %(message)s'
DEFAULT_LEVEL = logging.DEBUG

#w is overwrite, a is append. this creates a new log and it logs stuff in the logger file
class LogWrapper():
    def __init__(self, name, mode="w"):
        self.create_directory()       
        self.filename = './logs/' + name + '.log'
        self.logger = logging.getLogger(name)
        self.logger.setLevel(DEFAULT_LEVEL)
        
        file_handler = logging.FileHandler(self.filename,mode=mode)
        formatter = logging.Formatter(LOG_FORMAT,datefmt='%Y-%M-%D %H:%M:%S')
        file_handler.setFormatter(formatter)
        self.logger.addHandler(file_handler)
        self.logger.info('LogWrapper init() '+self.filename)

    def create_directory(self):
        path = './logs'
        if not os.path.exists(path):
            os.makedirs(path)

            
if __name__ == "__main__":
    #creates log file called test
    log = LogWrapper(name="Test")
    #adds to log file
    log.logger.debug("HELLO")

In [7]:
###create json file
data = {
    'GBP_JPY' :  { "pair" :  'GBP_JPY', "units": 1000, "pip": 0.01},
    'GBP_AUD' :  { "pair" :  'GBP_AUD', "units": 1000, "pip": 0.0001},
    'GBP_USD' :  { "pair" :  'GBP_USD', "units": 1000, "pip": 0.0001},
    'USD_CAD' :  { "pair" :  'USD_CAD', "units": 1000, "pip": 0.0001},
    'USD_JPY' :  { "pair" :  'USD_JPY', "units": 1000, "pip": 0.01},
    #'AUD_JPY' :  { "pair" :  'AUD_JPY', "units": 1000, "pip": 0.01},
    #'AUD_CAD' :  { "pair" :  'AUD_CAD', "units": 1000, "pip": 0.0001},
    #'EUR_USD' :  { "pair" :  'EUR_USD', "units": 1000, "pip": 0.0001},
    #'EUR_JPY' :  { "pair" :  'EUR_JPY', "units": 1000, "pip": 0.01},
    #'EUR_AUD' :  { "pair" :  'EUR_AUD', "units": 1000, "pip": 0.0001},
    #'CAD_JPY' :  { "pair" :  'CAD_JPY', "units": 1000, "pip": 0.01},
    #'XAU_USD' :  { "pair" :  'XAU_USD', "units": 1,    "pip": 0.1}
    
}

with open('settings1.json', 'w') as outfile:
    json.dump(data, outfile)
###

class Settings():
    def __init__(self, pair, units,pip):
        self.pair = pair
        self.units = units
        self.pip = pip

    def __repr__(self):
        return str(vars(self))
    
    #return details of json file above as settings class
    @classmethod
    def from_file_ob(cls, ob):
        return Settings(ob['pair'],ob['units'], ob['pip'])
    
    @classmethod
    def load_settings(cls):
        data = json.loads(open('settings1.json', 'r').read())
        return { k:cls.from_file_ob(v) for k,v in data.items() }
    
    #only get pairs from settings file
    @classmethod
    def get_pairs(cls):
        return list(cls.load_settings().keys())

if __name__ == "__main__":
    #[print(k,v) for k,v in Settings.load_settings().items()]
    #print(Settings.get_pairs())
    print(Settings.load_settings()['GBP_JPY'])
    

{'pair': 'GBP_JPY', 'units': 1000, 'pip': 0.01}


In [8]:
# if there is no last candle, we set the timing to 10 minutes before, so we do not miss this current first candle 
# when it ends
class Timing():
    def __init__(self,last_candle):
        self.last_candle = last_candle
        if self.last_candle is None:
            self.last_candle = time_utc()-dt.timedelta(minutes=5)
        #set to true when have new candle
        self.ready=False
    
    #when init, it will run repr, which prints out a dict of all self.items
    def __repr__(self):
        return str(vars(self))

In [9]:
BUY =1
SELL =-1
NONE = 0

class Technicals():
    def __init__(self,settings,api,pair,granularity,log = None):
        self.settings=settings
        self.settings_loaded= Settings.load_settings()
        self.api = api
        self.log = log
        self.pair = pair
        self.granularity = granularity
        
    def log_message(self,msg):
        if self.log is not None:
            self.log.logger.debug(msg)
    
    def fetch_candles(self,row_count,candle_time):
        status_code, df = self.api.fetch_candles(self.pair, count=row_count, granularity=self.granularity)
        if df is None:
            print('df error')
            self.log_message(f"Error fetching candles for pair:{self.pair} {candle_time}, df None")
            return None
        #when u fetch candles, the last candle must be same as the candle stored in Timing above
        elif df.iloc[-1].time>candle_time:
            print (status_code, 'iloc_error >')
            temp_df = pd.read_pickle('iloc_error.pkl')
            temp_df = temp_df.append(df)
            temp_df.to_pickle('iloc_error.pkl')
            self.log_message(f"Error fetching candles for pair:{self.pair} {candle_time} vs {df.iloc[-1].time}")
            return df
        
        elif df.iloc[-1].time<candle_time:
            print (status_code, 'iloc_error <')
            self.log_message('decision =0')
            self.log_message(f"Error fetching candles for pair:{self.pair} {candle_time} vs {df.iloc[-1].time}")
            return None
        
        else:
            return df
            
    #return 1 if buy and -1 if sell
    def process_candles(self,df):
    
         df = df[['time','mid_c','mid_o']].copy()
            
         df['PAIR']= self.pair

         df['prev_close_m2']= df.mid_c.shift(2)

         df['prev_close_m1']= df.mid_c.shift(1)

         df['prev_open_m2']= df.mid_o.shift(2)

         df['prev_open_m1']= df.mid_o.shift(1)

         df['prev_close_m30']= df.mid_c.shift(30)
        
         last = df.iloc[-1]

         decision = NONE
         sup_com_df = pd.read_pickle(f'com/{self.pair}_com_sup.pkl')
         res_com_df = pd.read_pickle(f'com/{self.pair}_com_resist.pkl')

         #
         sup_df = pd.read_pickle(f'{self.pair}_pivots_support.pkl')
         res_df = pd.read_pickle(f'{self.pair}_pivots_resist.pkl')

         for x in sup_df.index:
            upper_limit = sup_df['sup_level'][x] + self.settings_loaded[self.pair].pip*10
            lower_limit = sup_df['sup_level'][x] - self.settings_loaded[self.pair].pip*10
            if sup_df['sup_level'][x] not in sup_com_df.values:
                if last['prev_close_m2']<last['prev_open_m2'] and last['prev_close_m2']<upper_limit and last['prev_close_m2']>lower_limit:
                    if last['prev_close_m1']>last['prev_open_m1'] and last['mid_c']>last['mid_o'] and last['mid_c']>upper_limit:
                        if last['prev_close_m30']>last['mid_c']:
                            decision = BUY
                            self.add_to_com_sup(sup_df['dates'][x],sup_df['sup_level'][x])
                            self.log_message(f"decision details\n{sup_df['dates'][x]}: {sup_df['sup_level'][x]}")
                    
                            log_cols = ['time','PAIR','prev_close_m30','prev_open_m2','prev_close_m2','prev_open_m1','prev_close_m1','mid_o','mid_c']
                            temp_df = df[log_cols].tail(1)
                            temp_df["sure_level"]= sup_df['sup_level'][x]
                            temp_df['upper_limit']=upper_limit
                            temp_df['lower_limit']=lower_limit
                            temp_df['decision']=decision
                            temp_df1= pd.read_pickle('respect_sure_buy_log.pkl')
                            temp_df1 = temp_df1.append(temp_df)
                            temp_df1.to_pickle('respect_sure_buy_log.pkl')

         for x in res_df.index:
            upper_limit = res_df['res_level'][x] + self.settings_loaded[self.pair].pip*10
            lower_limit = res_df['res_level'][x] - self.settings_loaded[self.pair].pip*10
            if res_df['res_level'][x] not in res_com_df.values:
                if last['prev_close_m2']>last['prev_open_m2'] and last['prev_close_m2']<upper_limit and last['prev_close_m2']>lower_limit:
                    if last['prev_close_m1']<last['prev_open_m1'] and last['mid_c']<last['mid_o'] and last['mid_c']<lower_limit:
                        if last['prev_close_m30']<last['mid_c']:
                            decision = SELL
                            self.add_to_com_resist(res_df['dates'][x],res_df['res_level'][x])
                            self.log_message(f"decision details\n{res_df['dates'][x]}: {res_df['res_level'][x]}")

                            log_cols = ['time','PAIR','prev_close_m30','prev_open_m2','prev_close_m2','prev_open_m1','prev_close_m1','mid_o','mid_c']
                            temp_df = df[log_cols].tail(1).copy()
                            temp_df.reset_index(inplace=True,drop=True)
                            temp_df["sure_level"]= res_df['res_level'][x]
                            temp_df['upper_limit']=upper_limit
                            temp_df['lower_limit']=lower_limit
                            temp_df['decision']=decision
                            temp_df1= pd.read_pickle('respect_sure_buy_log.pkl')
                            temp_df1 = temp_df1.append(temp_df)
                            temp_df1.to_pickle('respect_sure_buy_log.pkl')
                    
         log_cols = ['time','PAIR','mid_o','mid_c','prev_close_m2','prev_close_m1','prev_open_m2','prev_open_m1']
         self.log_message(f"Trade_decision:{decision}")
         self.log_message(f"Processed_df\n{df[log_cols].tail(3)}")
         #
         return decision
    
    #get df and run process_candles
    def get_trade_decision(self, candle_time):
        self.log_message(f'get_trade_decision() pair:{self.pair}')
        
        df = self.fetch_candles(32,candle_time)
        
        if df is not None:
            return self.process_candles(df)
        
        return None
    
    def add_to_com_sup(self,date,sup_level):
        temp_df = pd.read_pickle(f'com/{self.pair}_com_sup.pkl')
        new_temp_df = pd.DataFrame({'date':[date],'sup_level':[sup_level],'date_bought':[dt.datetime.utcnow()]})
        temp_df = temp_df.append(new_temp_df)
        temp_df.to_pickle(f'com/{self.pair}_com_sup.pkl')  
        
    def add_to_com_resist(self,date,res_level):
        temp_df = pd.read_pickle(f'com/{self.pair}_com_resist.pkl')
        new_temp_df = pd.DataFrame({'date':[date],'res_level':[res_level],'date_bought':[dt.datetime.utcnow()]})
        temp_df = temp_df.append(new_temp_df)
        temp_df.to_pickle(f'com/{self.pair}_com_resist.pkl')
    
    
                                              

In [10]:
class TradeManager():
    def __init__(self,api,settings,log=None):
        self.api=api
        self.settings = settings
        self.settings_loaded= Settings.load_settings()
        self.log= log
        
    def log_message(self,msg):
        if self.log is not None:
            self.log.logger.debug(msg)
    
    def place_trades(self,trades_to_make):
        self.log_message(f'TradeManager:place_trades() {trades_to_make}')
        
        #original>
        #pairs = [x['pair'] for x in trades_to_make]
        
        # proposed change
        pairs = []
        open_trades, ok = self.api.open_trades()
        for x in trades_to_make:
          for y in open_trades:
             if x['pair']==y.instrument:
                if x['units']<0 and y.currentUnits>0:
                    pairs.append(y.instrument)
                elif x['units']>0 and y.currentUnits<0:
                    pairs.append(y.instrument)
        
        self.close_trades(pairs)
        self.create_trades(trades_to_make)
        
    #use oandaapi to place trade for each dict item in trades to make and return trade id
                
    def create_trades(self,trades_to_make):
        for t in trades_to_make:
            trade_id = self.api.place_trade(t['pair'],t['units'],take_profit=self.settings_loaded[t['pair']].pip*20, stop_loss=self.settings_loaded[t['pair']].pip*20)
            
            if trade_id is not None:
                self.log_message(f"TradeManager:Opened {trade_id} {t}")
            else:
                self.log_message(f"TradeManager:FAILED TO OPEN {t}")
                 
    def close_trades(self,pairs_to_close):
        open_trades, ok = self.api.open_trades()
        
        if ok==False:
            self.log_message('Error fetching open trades!')
            return
        #get the trade ids from the OandaTrade objects in open_trades, return its instruments and check if those are>
        #in pairs_to_close, which are the pairs we are looking to trade now
        ids_to_close =[x.trade_id for x in open_trades if x.instrument in pairs_to_close]
        
        self.log_message(f"TradeManager:close_trades() pairs_to_close:{pairs_to_close} ")
        self.log_message(f"TradeManager:close_trades() open_trades:{open_trades} ")
        self.log_message(f"TradeManager:close_trades() ids_to_close:{ids_to_close} ")
        
        # close trade and check status
        for t in ids_to_close:
            ok = self.api.close_trade(t)
            if ok == False:
                self.log_message(f"TradeManager:close_trades() {t} FAILED TO CLOSE ")
            else:
                self.log_message(f"TradeManager:close_trades() Closed")

In [11]:
GRANULARITY = 'M1'
SLEEP = 5.0

class TradingBot():
    
    def __init__(self):  
        #create a log everytime we start bot with printed settings
        self.log = LogWrapper("Bot")
        self.tech_log= LogWrapper("Technicals")
        self.trade_log= LogWrapper("Trade")
        self.trade_pairs = Settings.get_pairs()
        self.settings = Settings.load_settings()
        self.api = OandaAPI()
        self.trade_manager = TradeManager(self.api,self.settings,self.trade_log)
        #when init, self.timings is the dictionary of the timings of the timings of the pair (when started)
        self.timings = {p: Timing(self.api.last_complete_candle(p,GRANULARITY)) for p in self.trade_pairs}
        self.log_message(f"Bot started with\n{pprint.pformat(self.settings)}")
        self.log_message(f"Bot Timings\n{pprint.pformat(self.timings)}")
    
    # create this function, which runs the msg. during init, it will run the function above
    def log_message(self, msg):
        self.log.logger.debug(msg)
        
    # update timing of pair if current timing is more than previous timing. change status of pair to ready to trade
    def update_timings(self):
        for pair in self.trade_pairs:
            #fetch candles and check what is the timing and store it in current
            current = self.api.last_complete_candle(pair,GRANULARITY)
            self.timings[pair].ready = False
            #if current is more than the stored self.timings pair in the timings dict, update self.timings pair>
            #last timing
            if current> self.timings[pair].last_candle:
                self.timings[pair].ready = True  
                self.timings[pair].last_candle = current
                self.log_message(f'{pair}new candle {current}')
    
    def process_pairs(self):
        d = dt.datetime.utcnow().time()
        d1= dt.time(20,56)
        d2 = dt.time(21,0)

        if d>d1 and d<d2:
            trades, ok = self.api.open_trades()
            if ok == True:
                ok1 = [self.api.close_trade(t.trade_id) for t in trades]
                if ok1 == False:
                    self.log_message(f"TradeManager:close_trades() {t} FAILED TO CLOSE ")
                else:
                    self.log_message(f"TradeManager:close_trades() Closed at 5am")
                    
        trades_to_make = []
        
        for pair in self.trade_pairs:
            if self.timings[pair].ready == True:
                self.log_message(f'Ready to trade {pair}')
                techs = Technicals(self.settings[pair], self.api, pair, GRANULARITY, log=self.tech_log)
                decision = techs.get_trade_decision(self.timings[pair].last_candle)
                if decision == None:
                    print ('error decision 0')
                    decision = 0
                units = decision*self.settings[pair].units
                if units != 0:
                    trades_to_make.append({'pair':pair, 'units':units})
        
        if len(trades_to_make) >0:
            d = dt.datetime.utcnow().time()
            d1= dt.time(20,30)
            d2 = dt.time(22,30)
            if d<d1 or d> d2:
                print (trades_to_make)
                self.trade_manager.place_trades(trades_to_make)
    
    def run(self):
        while True:
            print ('update_timings()...', dt.datetime.now())
            self.update_timings()
            print ('process_pairs()...')
            self.process_pairs()
            time.sleep(SLEEP)

if __name__ == "__main__":
    b = TradingBot()
    b.run()

update_timings()... 2021-06-26 02:20:39.970278
process_pairs()...
update_timings()... 2021-06-26 02:20:46.408246
process_pairs()...
update_timings()... 2021-06-26 02:20:52.965408
process_pairs()...
update_timings()... 2021-06-26 02:20:59.410938
process_pairs()...
update_timings()... 2021-06-26 02:21:06.315561
process_pairs()...
update_timings()... 2021-06-26 02:21:14.073333
process_pairs()...
update_timings()... 2021-06-26 02:21:20.617009
process_pairs()...
update_timings()... 2021-06-26 02:21:27.063876
process_pairs()...
update_timings()... 2021-06-26 02:21:33.589473
process_pairs()...
update_timings()... 2021-06-26 02:21:40.078193
process_pairs()...
update_timings()... 2021-06-26 02:21:53.824870
process_pairs()...
update_timings()... 2021-06-26 02:22:00.439638
process_pairs()...
update_timings()... 2021-06-26 02:22:06.901931
process_pairs()...
update_timings()... 2021-06-26 02:22:14.682095
process_pairs()...
update_timings()... 2021-06-26 02:22:21.084897
process_pairs()...
update_tim

KeyboardInterrupt: 